#### Importing libraries

In [1]:
import xlwings as xw
from datetime import datetime
import openpyxl
from openpyxl.utils import get_column_letter
import numpy as np
import re
import pandas as pd


#### Diagnosis 1

##### Opening files

In [2]:
#Reading files
user = input("Enter your laptop user: ")
# Get the current date
current_date = datetime.now().strftime("%Y%m%d")

gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day}.xlsx'.format(user=user, çurrent_day=current_date))
sec_dep_det = xw.Book(r'C:\Users\{user}\Downloads\security_deposit_funds_detail-{çurrent_day}.xlsx'.format(user=user,çurrent_day=current_date))

# Indicating worksheet
gl_worksheet = gl.sheets['Sheet1']
sec_dep_det_worksheet = sec_dep_det.sheets['Sheet1']

##### Including adjusted payee names

In [3]:
# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])
gl.sheets['Sheet1'].range('E{}'.format(first_row_of_values - 2)).value = 'Payee'





##### Adding net column (debt - credit)

In [4]:
# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 
gl.sheets['Sheet1'].range('J{}'.format(first_row_of_values - 2)).value = 'Net'


##### Funds detail tenant name

In [5]:
sec_dep_det.sheets['Sheet1'].api.Columns(3).Insert() # Inserting column
sec_dep_det.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for tenant names



# Defining rows to calculate values
first_row_of_values_sec_dep = sec_dep_det.sheets['Sheet1'].range('a'+ str(sec_dep_det.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 1 # first row of values
last_row_of_values_sec_dep = sec_dep_det.sheets['Sheet1'].range('a'+ str(sec_dep_det.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row  # last row of values


# Read the data into a pandas DataFrame
column_data = sec_dep_det.sheets['Sheet1'].range('b{}:b{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value
df = pd.DataFrame(column_data, columns=['Names'])

# Rearrange the names efficiently using pandas
df['Names'] = df['Names'].str.split(', ').str[::-1].str.join(' ')
df['Names'] = df['Names'].str.replace(r'\s+', '', regex=True)

# Write the updated data back to Excel
sec_dep_det.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value = dict(df['Names'])


##### Creating pivot table

In [6]:
# Open a new Excel workbook
diagnostics_1_wb = xw.Book()

# Property Name
diagnostics_1_wb.sheets['Sheet1'].range('a1:g1').api.Font.Bold = True
gl.sheets['Sheet1'].range('m{}:m{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('A2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('A1').value = 'Property Name'

# Payee

gl.sheets['Sheet1'].range('e{}:e{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('b2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('b1').value = 'Payee'

# SUM of net
gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).formula = "=SUMIFS($J${first_row}:$J${last_row},$E${first_row}:$E${last_row},E{first_row},$M${first_row}:$M${last_row},M{first_row})".format(first_row=first_row_of_values, last_row= last_row_of_values)
values_list = gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).value
result_dict = {index: value for index, value in enumerate(values_list)}
diagnostics_1_wb.sheets['Sheet1'].range('c2').value = result_dict
diagnostics_1_wb.sheets['Sheet1'].range('c:c').api.Delete() # Deleting column C
diagnostics_1_wb.sheets['Sheet1'].range('c1').value = 'Sum of Net'

# SD FUNDS DETAIL
# diagnostics_1_wb last row
last_row = diagnostics_1_wb.sheets[0].range('A' + str(diagnostics_1_wb.sheets[0].cells.last_cell.row)).end('up').row

# formula base string
formula_base = (
    '=SUMIFS(\'[security_deposit_funds_detail-{current_date}.xlsx]Sheet1\'!$H${first_row_of_values_sec_dep}:$H${last_row_of_values_sec_dep},'
    '\'[security_deposit_funds_detail-{current_date}.xlsx]Sheet1\'!$I${first_row_of_values_sec_dep}:$I${last_row_of_values_sec_dep},'
    '{{}},'
    '\'[security_deposit_funds_detail-{current_date}.xlsx]Sheet1\'!$D${first_row_of_values_sec_dep}:$D${last_row_of_values_sec_dep},{{}})'
).format(current_date=current_date, first_row_of_values_sec_dep=first_row_of_values_sec_dep, last_row_of_values_sec_dep=last_row_of_values_sec_dep)

# Generate a list of formulas for the entire range
formulas = [formula_base.format(f'A{i}', f'B{i}') for i in range(1, last_row + 1)]

# Set the formulas for the entire column A at once
column_range = diagnostics_1_wb.sheets[0].range(f'd1:d{last_row}')
column_range.options(transpose=True).formula = formulas
column_range.options(transpose=True).value = column_range.value
diagnostics_1_wb.sheets['Sheet1'].range('d1').value = 'SD Funds Detail'


# Difference
# Get the data from Excel into a DataFrame
sheet = diagnostics_1_wb.sheets[0]  
data_range = sheet.range('c1').expand('table')  # The data starts from C1 and is structured as a table
df = data_range.options(pd.DataFrame, index=False, header=True).value  # Read data into DataFrame

# Calculate the difference between columns 'Sum of Net' and 'SD Funds Detail'
df['Difference'] = round(df['Sum of Net'] + df['SD Funds Detail'])

# Write the results back to Excel
start_cell = data_range.last_cell.offset(row_offset=2, column_offset=0)  
sheet.range(diagnostics_1_wb.sheets[0]['E1']).value = df['Difference']  # Write DataFrame back to Excel, starting in E1
# Delete the entire column
diagnostics_1_wb.sheets[0].api.Columns('e').Delete()



# Save the workbook as an Excel file
diagnostics_1_wb.save(r'C:\Users\{user}\Downloads\diagnosis_1.xlsx'.format(user=user))

# Close the workbook
diagnostics_1_wb.close()
gl.close()
sec_dep_det.close()




#### Diagnosis 2


In [7]:
gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day} (1).xlsx'.format(user=user, çurrent_day=current_date))
gl_worksheet = gl.sheets['Sheet1']


# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])
gl.sheets['Sheet1'].range('E{}'.format(first_row_of_values - 2)).value = 'Payee'

# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 
gl.sheets['Sheet1'].range('J{}'.format(first_row_of_values - 2)).value = 'Net'


# Open a new Excel workbook
diagnostics_1_wb = xw.Book()

# Property Name
diagnostics_1_wb.sheets['Sheet1'].range('a1:g1').api.Font.Bold = True
gl.sheets['Sheet1'].range('m{}:m{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('A2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('A1').value = 'Property Name'

# Payee

gl.sheets['Sheet1'].range('e{}:e{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('b2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('b1').value = 'Payee'

# SUM of net
gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).formula = "=SUMIFS($J${first_row}:$J${last_row},$E${first_row}:$E${last_row},E{first_row},$M${first_row}:$M${last_row},M{first_row})".format(first_row=first_row_of_values, last_row= last_row_of_values)
values_list = gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).value
result_dict = {index: value for index, value in enumerate(values_list)}
diagnostics_1_wb.sheets['Sheet1'].range('c2').value = result_dict
diagnostics_1_wb.sheets['Sheet1'].range('c:c').api.Delete() # Deleting column C
diagnostics_1_wb.sheets['Sheet1'].range('c1').value = 'Sum of Net'


# Save the workbook as an Excel file
diagnostics_1_wb.save(r'C:\Users\{user}\Downloads\diagnosis_2.xlsx'.format(user=user))

# Close the workbook
gl.close()
diagnostics_1_wb.close()


#### Diagnosis 3

##### Opening files

In [8]:
gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day} (2).xlsx'.format(user=user, çurrent_day=current_date))
bill_detail = xw.Book(r'C:\Users\{user}\Downloads\bill_detail-{çurrent_day}.xlsx'.format(user=user,çurrent_day=current_date))

# Indicating worksheet
gl_worksheet = gl.sheets['Sheet1']
bill_detail_worksheet = bill_detail.sheets['Sheet1']

##### Including adjusted payee names

In [9]:
# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])
gl.sheets['Sheet1'].range('E{}'.format(first_row_of_values - 2)).value = 'Payee'


##### Adding net column (debt - credit)

In [10]:
# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 
gl.sheets['Sheet1'].range('J{}'.format(first_row_of_values - 2)).value = 'Net'

##### Bill detail tenant name

In [11]:
bill_detail.sheets['Sheet1'].api.Columns(8).Insert() # Inserting column
bill_detail.sheets['Sheet1'].api.Columns(9).Insert() # Inserting another column to include numerical index for tenant names



# Defining rows to calculate values
first_row_of_values_bill_detail = bill_detail.sheets['Sheet1'].range('b'+ str(bill_detail.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row + 1 # first row of values
last_row_of_values_bill_detail = bill_detail.sheets['Sheet1'].range('b'+ str(bill_detail.sheets['Sheet1'].cells.last_cell.row)).end('up').row  # last row of values


# Read the data into a pandas DataFrame
column_data = bill_detail.sheets['Sheet1'].range('g{}:g{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value
df = pd.DataFrame(column_data, columns=['Names'])

# Rearrange the names efficiently using pandas
df['Names'] = df['Names'].str.split(', ').str[::-1].str.join(' ')
df['Names'] = df['Names'].str.replace(r'\s+', '', regex=True)

# Write the updated data back to Excel
bill_detail.sheets['Sheet1'].range('h{}:h{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value = dict(df['Names'])

##### Creating pivot table

In [12]:
# Open a new Excel workbook
diagnostics_1_wb = xw.Book()

# Property Name
diagnostics_1_wb.sheets['Sheet1'].range('a1:g1').api.Font.Bold = True
gl.sheets['Sheet1'].range('m{}:m{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('A2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('A1').value = 'Property Name'

# Payee

gl.sheets['Sheet1'].range('e{}:e{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('b2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('b1').value = 'Payee'

# SUM of net
gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).formula = "=SUMIFS($J${first_row}:$J${last_row},$E${first_row}:$E${last_row},E{first_row},$M${first_row}:$M${last_row},M{first_row})".format(first_row=first_row_of_values, last_row= last_row_of_values)
values_list = gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).value
result_dict = {index: value for index, value in enumerate(values_list)}
diagnostics_1_wb.sheets['Sheet1'].range('c2').value = result_dict
diagnostics_1_wb.sheets['Sheet1'].range('c:c').api.Delete() # Deleting column C
diagnostics_1_wb.sheets['Sheet1'].range('c1').value = 'Sum of Net'

# SD BILL DETAIL
# diagnostics_1_wb last row
last_row = diagnostics_1_wb.sheets[0].range('A' + str(diagnostics_1_wb.sheets[0].cells.last_cell.row)).end('up').row


# formula base string
formula_base = (
    '=SUMIFS(\'[bill_detail-{current_date}.xlsx]Sheet1\'!$k${first_row_of_values_bill_detail}:$k${last_row_of_values_bill_detail},'
    '\'[bill_detail-{current_date}.xlsx]Sheet1\'!$p${first_row_of_values_bill_detail}:$p${last_row_of_values_bill_detail},'
    '{{}},'
    '\'[bill_detail-{current_date}.xlsx]Sheet1\'!$i${first_row_of_values_bill_detail}:$i${last_row_of_values_bill_detail},{{}})'
).format(current_date=current_date, first_row_of_values_bill_detail=first_row_of_values_bill_detail, last_row_of_values_bill_detail=last_row_of_values_bill_detail)

# Generate a list of formulas for the entire range
formulas = [formula_base.format(f'A{i}', f'B{i}') for i in range(1, last_row + 1)]


# Set the formulas for the entire column A at once
column_range = diagnostics_1_wb.sheets[0].range(f'd1:d{last_row}')
column_range.options(transpose=True).formula = formulas
column_range.options(transpose=True).value = column_range.value
diagnostics_1_wb.sheets['Sheet1'].range('d1').value = 'Bill Detail'


# Difference
# Get the data from Excel into a DataFrame
sheet = diagnostics_1_wb.sheets[0]  
data_range = sheet.range('c1').expand('table')  # The data starts from C1 and is structured as a table
df = data_range.options(pd.DataFrame, index=False, header=True).value  # Read data into DataFrame

# Calculate the difference between columns 'Sum of Net' and 'SD Funds Detail'
df['Difference'] = round(df['Sum of Net'] + df['Bill Detail'])

# Write the results back to Excel
start_cell = data_range.last_cell.offset(row_offset=2, column_offset=0)  
sheet.range(diagnostics_1_wb.sheets[0]['E1']).value = df['Difference']  # Write DataFrame back to Excel, starting in E1
# Delete the entire column
diagnostics_1_wb.sheets[0].api.Columns('e').Delete()



# Save the workbook as an Excel file
diagnostics_1_wb.save(r'C:\Users\{user}\Downloads\diagnosis_3.xlsx'.format(user=user))

# Close the workbook
diagnostics_1_wb.close()
gl.close()
bill_detail.close()


#### Diagnosis 4

##### Opening files

In [13]:

gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day} (3).xlsx'.format(user=user, çurrent_day=current_date))
delinquency = xw.Book(r'C:\Users\{user}\Downloads\delinquency-{çurrent_day}.xlsx'.format(user=user,çurrent_day=current_date))

# Indicating worksheet
gl_worksheet = gl.sheets['Sheet1']
delinquency_worksheet = delinquency.sheets['Sheet1']

##### Including adjusted payee names

In [14]:
# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])
gl.sheets['Sheet1'].range('E{}'.format(first_row_of_values - 2)).value = 'Payee'


##### Adding net column (debt - credit)

In [15]:
# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 
gl.sheets['Sheet1'].range('J{}'.format(first_row_of_values - 2)).value = 'Net'

##### Delinquency tenant name

In [16]:
delinquency.sheets['Sheet1'].api.Columns(3).Insert() # Inserting column
delinquency.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for tenant names



# Defining rows to calculate values
first_row_of_values_deliquency = delinquency.sheets['Sheet1'].range('a'+ str(delinquency.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 1 # first row of values
last_row_of_values_deliquency = delinquency.sheets['Sheet1'].range('a'+ str(delinquency.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row  # last row of values

# Read the data into a pandas DataFrame
column_data = delinquency.sheets['Sheet1'].range('b{}:b{}'.format(first_row_of_values_deliquency, last_row_of_values_deliquency)).value
df = pd.DataFrame(column_data, columns=['Names'])

# Rearrange the names efficiently using pandas
df['Names'] = df['Names'].str.split(', ').str[::-1].str.join(' ')
df['Names'] = df['Names'].str.replace(r'\s+', '', regex=True)

# Write the updated data back to Excel
delinquency.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values_deliquency, last_row_of_values_deliquency)).value = dict(df['Names'])

##### Creating pivot table

In [17]:
# Open a new Excel workbook
diagnostics_1_wb = xw.Book()

# Property Name
diagnostics_1_wb.sheets['Sheet1'].range('a1:g1').api.Font.Bold = True
gl.sheets['Sheet1'].range('m{}:m{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('A2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('A1').value = 'Property Name'

# Payee

gl.sheets['Sheet1'].range('e{}:e{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('b2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('b1').value = 'Payee'



# # SUM of net
# defining columns to calculate values
e_values = gl.sheets['Sheet1']['e{}:e{}'.format(first_row_of_values, last_row_of_values)].value
m_values = gl.sheets['Sheet1']['m{}:m{}'.format(first_row_of_values, last_row_of_values)].value
j_values = gl.sheets['Sheet1']['j{}:j{}'.format(first_row_of_values, last_row_of_values)].value

# DataFrame using the retrieved values
data = {
    'E': e_values,
    'M': m_values,
    'J': j_values
}

df = pd.DataFrame(data)

# Calculate the grouped sum
result = df.groupby(['E', 'M'])['J'].transform('sum')

# Add the result as a new column at the end of the DataFrame
df['SUMIFS_Result'] = result

# Including final result in excel file column "O"
diagnostics_1_wb.sheets['Sheet1']['c2'].value = dict(df['SUMIFS_Result'])
diagnostics_1_wb.sheets[0].api.Columns('c').Delete()
diagnostics_1_wb.sheets['Sheet1'].range('c1').value = 'SUM of net'



# DELINQUENCY DETAIL
# defining columns to calculate values
d_values = delinquency.sheets['Sheet1']['d{}:d{}'.format(first_row_of_values_deliquency, last_row_of_values_deliquency)].value
g_values = delinquency.sheets['Sheet1']['g{}:g{}'.format(first_row_of_values_deliquency, last_row_of_values_deliquency)].value
h_values = delinquency.sheets['Sheet1']['h{}:h{}'.format(first_row_of_values_deliquency, last_row_of_values_deliquency)].value

# DataFrame using the retrieved values
data = {
    'a': d_values,
    'b': g_values,
    'c': h_values
}

df_1 = pd.DataFrame(data)

# Calculate the grouped sum
result = df_1.groupby(['a', 'b'])['c'].transform('sum')

# Add the result as a new column at the end of the DataFrame
df_1['SUMIFS_Result'] = result


last_row = diagnostics_1_wb.sheets[0].range('A' + str(diagnostics_1_wb.sheets[0].cells.last_cell.row)).end('up').row

a_values = diagnostics_1_wb.sheets['Sheet1']['a{}:a{}'.format(2, last_row)].value
b_values = diagnostics_1_wb.sheets['Sheet1']['b{}:b{}'.format(2, last_row)].value

# DataFrame using the retrieved values
data = {
    'a': b_values,
    'b': a_values
}

df_2 = pd.DataFrame(data)



# Creating a dictionary-like structure from df1
mapping_dict = dict(zip(zip(df_1['a'], df_1['b']), df_1['SUMIFS_Result']))

# Mapping the values from df1 to df2 based on 'G' and 'D' columns
df_2['Total Delinquency'] = df_2.apply(lambda row: mapping_dict.get((row['a'], row['b']), None), axis=1)
diagnostics_1_wb.sheets['Sheet1']['D2'].value = dict(df_2['Total Delinquency'])
diagnostics_1_wb.sheets[0].api.Columns('D').Delete()
diagnostics_1_wb.sheets['Sheet1'].range('D1').value = 'Delinquency'
diagnostics_1_wb.sheets['Sheet1'].range('D1').api.Font.Bold = True



# Difference
# Get the data from Excel into a DataFrame
sheet = diagnostics_1_wb.sheets[0]  
data_range = sheet.range('c1').expand('table')  # The data starts from C1 and is structured as a table
df = data_range.options(pd.DataFrame, index=False, header=True).value  # Read data into DataFrame

# Calculate the difference between columns 'Sum of Net' and 'SD Funds Detail'
df['Difference'] = round(df['SUM of net'] + df['Delinquency'])

# Write the results back to Excel
start_cell = data_range.last_cell.offset(row_offset=2, column_offset=0)  
sheet.range(diagnostics_1_wb.sheets[0]['E1']).value = df['Difference']  # Write DataFrame back to Excel, starting in E1
# Delete the entire column
diagnostics_1_wb.sheets[0].api.Columns('e').Delete()
diagnostics_1_wb.sheets['Sheet1'].range('e1').api.Font.Bold = True



# Save the workbook as an Excel file
diagnostics_1_wb.save(r'C:\Users\{user}\Downloads\diagnosis_4.xlsx'.format(user=user))

# Close the workbook
diagnostics_1_wb.close()
gl.close()
delinquency.close()





#### Notes

- No cambiarle el nombre a los archivos que se descargan de appfolio

- no tener en la carpeta de descargas archivos previos a este proceso con los siguientes nombres
general_ledger-20231110

    - security_deposit_funds_detail-20231110

    - delinquency-20231110

    - bill_detail-20231214